# Feasibility Check Demo: Full Model Integration

This notebook demonstrates the **Feasibility & Minimum Stock Check** logic using the full **Drawing-to-ERP** workflow.

**The Workflow:**
1.  **Input**: A `BillOfMaterials` object (as if extracted from a customer drawing). 
    -   Contains *external* data: Part Numbers (e.g., `9142083116`), Descriptions, Quantities.
    -   Does *not* contain Xentral Internal IDs.
2.  **Resolution**: The `check_feasibility` tool automatically resolves external Part Numbers to Xentral Internal IDs.
3.  **Stock Check**: It validates stock levels and minimum stock thresholds for the resolved items.

**Test Subject (Live Data):**
-   **Drawing Part Number**: `9142083116`
-   **Internal Xentral ID**: `3`
-   **Live Stock**: ~81 units
-   **Minimum Stock**: ~20 units

In [1]:
import sys
import os
import json
from pathlib import Path
from dotenv import load_dotenv

# Setup Paths
repo_root = Path("d:/Users/aaron/vsc/kako-ai")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Load .env
env_path = repo_root / ".env"
load_dotenv(env_path)

print(f"Loaded .env from: {env_path}")

# Import Models & Tool
from backend.src.models import BillOfMaterials, BOMItem
from backend.src.tools.demand_analysis.bom import check_feasibility

Loaded .env from: d:\Users\aaron\vsc\kako-ai\.env


## 1. Constructing the BOM Model
We simulate the output of our PDF Extraction Agent. We created a BOM object containing the **External Part Number** `9142083116`.

In [2]:
# Define the BOM structure extracted from a drawing
drawing_bom = BillOfMaterials(
    items=[
        BOMItem(
            item_nr="9142083116",       # External Part Number (from drawing)
            description="Test Connector", 
            quantity=1.0,               # Requires 1 per unit produced
            unit="pcs"
        )
    ]
)

print("Constructed Drawing BOM Model:")
print(drawing_bom)

Constructed Drawing BOM Model:
title=None items=[BOMItem(part_number='1', quantity=1.0, item_nr='9142083116', description='Test Connector', unit='pcs', xentral_number=None)]


In [3]:
def run_full_model_demo(bom_model: BillOfMaterials, order_qty: int, label: str):
    print(f"\n--- SCENARIO: {label} ---")
    print(f"Producing {order_qty} units based on Drawing BOM...")
    
    # Execute Check
    # The tool will autonomously resolve '9142083116' -> ID '3' and check stock.
    result_json = check_feasibility(bom_model, order_amount=order_qty)
    result = json.loads(result_json)
    
    # Parse details for display
    feasible = result.get("feasible")
    items = result.get("details", [])
    
    if items:
        item = items[0]
        print(f"Resolved Item: {item.get('name')} (Part #: {item.get('part_number')})")
        print(f"Stock Status:  {item.get('in_stock')} available (Min Required: {item.get('min_stock')})")
        print(f"Feasibility:   {feasible}")
        
        if item.get("warning"):
            print(f"⚠️ WARNING: {item.get('warning')}")
        else:
            print("✅ No Warnings")
    else:
        print("Error: No items resolved.")

## 2. Scenario: Safe Order
Producing **50 units**.
-   Required: 50 * 1 = 50
-   Stock: 81
-   Remaining: 31 (> 20 Min)
-   **Expected**: Feasible, No Warnings.

In [4]:
run_full_model_demo(drawing_bom, order_qty=50, label="Safe Order")


--- SCENARIO: Safe Order ---
Producing 50 units based on Drawing BOM...
Resolved Item: Test Connector (Part #: 9142083116)
Stock Status:  81 available (Min Required: 20)
Feasibility:   True
✅ No Warnings


## 3. Scenario: Warning Trigger
Producing **65 units**.
-   Required: 65 * 1 = 65
-   Stock: 81
-   Remaining: 16 (< 20 Min)
-   **Expected**: Feasible, but with **Warning**.

In [5]:
run_full_model_demo(drawing_bom, order_qty=65, label="Warning Trigger")


--- SCENARIO: Warning Trigger ---
Producing 65 units based on Drawing BOM...
Resolved Item: Test Connector (Part #: 9142083116)
Stock Status:  81 available (Min Required: 20)
Feasibility:   True
⚠️ WARNING: Stock will fall below minimum! (Remaining: 16.0, Min: 20)


## 4. Scenario: Infeasible Order
Producing **100 units**.
-   Required: 100 * 1 = 100
-   Stock: 81
-   **Expected**: Infeasible (Not enough stock).

In [6]:
run_full_model_demo(drawing_bom, order_qty=100, label="Infeasible Order")


--- SCENARIO: Infeasible Order ---
Producing 100 units based on Drawing BOM...
Resolved Item: Test Connector (Part #: 9142083116)
Stock Status:  81 available (Min Required: 20)
Feasibility:   False
✅ No Warnings
